## Imports

In [1]:
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader


if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

## Settings and Dataset

In [2]:
##########################
### SETTINGS
##########################

# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Hyperparameters
random_seed = 1
learning_rate = 0.001
num_epochs = 10
batch_size = 128

# Architecture
num_features = 784
num_classes = 10


##########################
### MNIST DATASET
##########################

# Note transforms.ToTensor() scales input images
# to 0-1 range
train_dataset = datasets.CIFAR10(root='data', 
                                 train=True, 
                                 transform=transforms.ToTensor(),
                                 download=True)

test_dataset = datasets.CIFAR10(root='data', 
                                train=False, 
                                transform=transforms.ToTensor())


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size, 
                         shuffle=False)

# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

cuda:0
Files already downloaded and verified
Image batch dimensions: torch.Size([128, 3, 32, 32])
Image label dimensions: torch.Size([128])


## Model

In [3]:
##########################
### MODEL
##########################


class VGG5(torch.nn.Module):

    def __init__(self, num_features, num_classes):
        super(VGG5, self).__init__()
        
        # calculate same padding:
        # (w - k + 2*p)/s + 1 = o
        # => p = (s(o-1) - w + k)/2
        
        self.block_1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.Tanh(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),  
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.Tanh(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),  
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),  
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
            
        self.classifier = nn.Sequential(
            nn.Linear(256, 4096),
            nn.Tanh(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.Tanh(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes),
        )
            
        for m in self.modules():
            if isinstance(m, torch.nn.Conv2d) or isinstance(m, torch.nn.Linear):
                nn.init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='tanh')
                if m.bias is not None:
                    m.bias.detach().zero_()
                    
        #self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        
        
    def forward(self, x):
        x = self.block_1(x)
        #x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        logits = self.classifier(x)
        probas = F.softmax(logits, dim=1)

        return logits, probas

    
torch.manual_seed(random_seed)
model = VGG5(num_features=num_features,
              num_classes=num_classes)

model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

## Training

In [4]:
def compute_accuracy(model, data_loader):
    model.eval()
    correct_pred, num_examples = 0, 0
    for i, (features, targets) in enumerate(data_loader):
            
        features = features.to(device)
        targets = targets.to(device)

        logits, probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100


def compute_epoch_loss(model, data_loader):
    model.eval()
    curr_loss, num_examples = 0., 0
    with torch.no_grad():
        for features, targets in data_loader:
            features = features.to(device)
            targets = targets.to(device)
            logits, probas = model(features)
            loss = F.cross_entropy(logits, targets, reduction='sum')
            num_examples += targets.size(0)
            curr_loss += loss

        curr_loss = curr_loss / num_examples
        return curr_loss
    
    

start_time = time.time()
for epoch in range(num_epochs):
    
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.to(device)
        targets = targets.to(device)
            
        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %04d/%04d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_loader), cost))

    model.eval()
    with torch.set_grad_enabled(False): # save memory during inference
        print('Epoch: %03d/%03d | Train: %.3f%% |  Loss: %.3f' % (
              epoch+1, num_epochs, 
              compute_accuracy(model, train_loader),
              compute_epoch_loss(model, train_loader)))


    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
    
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

Epoch: 001/010 | Batch 0000/0391 | Cost: 3.1044
Epoch: 001/010 | Batch 0050/0391 | Cost: 2.4969
Epoch: 001/010 | Batch 0100/0391 | Cost: 2.4533
Epoch: 001/010 | Batch 0150/0391 | Cost: 2.2711
Epoch: 001/010 | Batch 0200/0391 | Cost: 2.1284
Epoch: 001/010 | Batch 0250/0391 | Cost: 1.8992
Epoch: 001/010 | Batch 0300/0391 | Cost: 2.0525
Epoch: 001/010 | Batch 0350/0391 | Cost: 1.9241
Epoch: 001/010 | Train: 33.770% |  Loss: 1.844
Time elapsed: 0.59 min
Epoch: 002/010 | Batch 0000/0391 | Cost: 1.8764
Epoch: 002/010 | Batch 0050/0391 | Cost: 1.8957
Epoch: 002/010 | Batch 0100/0391 | Cost: 1.8650
Epoch: 002/010 | Batch 0150/0391 | Cost: 1.7790
Epoch: 002/010 | Batch 0200/0391 | Cost: 1.9758
Epoch: 002/010 | Batch 0250/0391 | Cost: 1.7771
Epoch: 002/010 | Batch 0300/0391 | Cost: 1.9739
Epoch: 002/010 | Batch 0350/0391 | Cost: 1.8201
Epoch: 002/010 | Train: 39.058% |  Loss: 1.697
Time elapsed: 1.12 min
Epoch: 003/010 | Batch 0000/0391 | Cost: 1.6839
Epoch: 003/010 | Batch 0050/0391 | Cost: 1.8

## Evaluation

In [5]:
with torch.set_grad_enabled(False): # save memory during inference
    print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))

Test accuracy: 48.55%
